In [2]:
import json
import time
import requests
import jsonlines

# URL di Ollama locale
OLLAMA_URL = "http://localhost:11434/api/generate"

# Nome del modello creato con:
# ollama create bari-translator -f Modelfile
MODEL_NAME = "bari-translator"


In [3]:
italian_texts = [
    "Il Borgo Antico di Bari è il cuore storico della città, con vicoli stretti e chiese medievali.",
    "La Basilica di San Nicola è una meta di pellegrinaggio per fedeli da tutto il mondo.",
    "Il Castello Svevo di Bari è una fortezza costruita in epoca normanna.",
    "La focaccia barese è uno dei cibi più iconici della tradizione locale.",
    "Il lungomare di Bari è perfetto per una passeggiata al tramonto."
]


In [ ]:
def translate_with_ollama(text):
    # Costruisci prompt minimale compatibile col Modelfile
    prompt = f"<input>\n{text.strip()}\n</input>"

    response = requests.post(
        OLLAMA_URL,
        json={
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False
        }
    )

    if response.status_code != 200:
        raise Exception(f"Errore Ollama: {response.text}")

    raw_output = response.json().get("response", "")

    # Estrai solo English e Russian
    return extract_translations(raw_output)



In [ ]:
import re

def extract_translations(output):
    """
    Ritorna solo:
      english = ...
      russian = ...
    Ignorando tutto il resto che il modello potrebbe generare.
    """

    # Pattern robusti per blocchi English/Russian
    en = re.search(r"English\s*:\s*(.*?)(?=\n\s*Russian\s*:|\Z)", output, re.DOTALL | re.IGNORECASE)
    ru = re.search(r"Russian\s*:\s*(.*?)(?=\Z)", output, re.DOTALL | re.IGNORECASE)

    english = en.group(1).strip() if en else ""
    russian = ru.group(1).strip() if ru else ""

    return {
        "english": english,
        "russian": russian
    }


In [6]:
def extract_translations(llm_output):
    english = ""
    russian = ""

    lines = llm_output.splitlines()
    current = None

    for line in lines:
        l = line.strip()

        if l.lower().startswith("english"):
            current = "en"
            continue
        if l.lower().startswith("russian") or "рус" in l.lower():
            current = "ru"
            continue

        if current == "en":
            english += l + " "
        elif current == "ru":
            russian += l + " "

    return english.strip(), russian.strip()


In [7]:
output_file = "bari_dataset.jsonl"

with jsonlines.open(output_file, mode='w') as writer:
    for idx, text in enumerate(italian_texts):
        print(f"[{idx+1}/{len(italian_texts)}] Traduco...")

        try:
            llm_output = translate_with_ollama(text)
            en, ru = extract_translations(llm_output)

            entry = {
                "id": f"bari_{idx:04d}",
                "italian": text,
                "english": en,
                "russian": ru
            }

            writer.write(entry)

        except Exception as e:
            print("Errore:", e)

        time.sleep(0.3)

print("\nDataset generato:", output_file)


[1/5] Traduco...
Errore: Errore Ollama: {"error":"model 'bari-translator' not found"}
[2/5] Traduco...
Errore: Errore Ollama: {"error":"model 'bari-translator' not found"}
[3/5] Traduco...
Errore: Errore Ollama: {"error":"model 'bari-translator' not found"}
[4/5] Traduco...
Errore: Errore Ollama: {"error":"model 'bari-translator' not found"}
[5/5] Traduco...
Errore: Errore Ollama: {"error":"model 'bari-translator' not found"}

Dataset generato: bari_dataset.jsonl
